<a href="https://colab.research.google.com/github/Satwikram/Deep-Learning-Notebooks/blob/master/Basic%20Regression/Fuel_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [135]:
"""
@Author: Satwik Ram K

Basic Regression using Tensorflow

"""
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [136]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

2.2.0


In [137]:
from google.colab import files

files.upload()


Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"satwikram","key":"06c98063bdd0e68efaf2312bfe2c72df"}'}

In [138]:
! mkdir ~/.kaggle


! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json


mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [139]:
! kaggle datasets download -d uciml/autompg-dataset

autompg-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [140]:
! unzip /content/autompg-dataset.zip

Archive:  /content/autompg-dataset.zip
replace auto-mpg.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [201]:
dataset = pd.read_csv('/content/auto-mpg.csv')

In [202]:
dataset.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino


In [203]:
colum_names = dataset.columns

In [204]:
colum_names

Index(['mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'model year', 'origin', 'car name'],
      dtype='object')

Clean the data

In [205]:
dataset.isna().sum()

mpg             0
cylinders       0
displacement    0
horsepower      0
weight          0
acceleration    0
model year      0
origin          0
car name        0
dtype: int64

In [206]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    398 non-null    object 
 4   weight        398 non-null    int64  
 5   acceleration  398 non-null    float64
 6   model year    398 non-null    int64  
 7   origin        398 non-null    int64  
 8   car name      398 non-null    object 
dtypes: float64(3), int64(4), object(2)
memory usage: 28.1+ KB


In [207]:
dataset['horsepower']

0      130
1      165
2      150
3      150
4      140
      ... 
393     86
394     52
395     84
396     79
397     82
Name: horsepower, Length: 398, dtype: object

In [208]:
dataset['car name'].unique()

array(['chevrolet chevelle malibu', 'buick skylark 320',
       'plymouth satellite', 'amc rebel sst', 'ford torino',
       'ford galaxie 500', 'chevrolet impala', 'plymouth fury iii',
       'pontiac catalina', 'amc ambassador dpl', 'dodge challenger se',
       "plymouth 'cuda 340", 'chevrolet monte carlo',
       'buick estate wagon (sw)', 'toyota corona mark ii',
       'plymouth duster', 'amc hornet', 'ford maverick', 'datsun pl510',
       'volkswagen 1131 deluxe sedan', 'peugeot 504', 'audi 100 ls',
       'saab 99e', 'bmw 2002', 'amc gremlin', 'ford f250', 'chevy c20',
       'dodge d200', 'hi 1200d', 'chevrolet vega 2300', 'toyota corona',
       'ford pinto', 'plymouth satellite custom', 'ford torino 500',
       'amc matador', 'pontiac catalina brougham', 'dodge monaco (sw)',
       'ford country squire (sw)', 'pontiac safari (sw)',
       'amc hornet sportabout (sw)', 'chevrolet vega (sw)',
       'pontiac firebird', 'ford mustang', 'mercury capri 2000',
       'opel 1900'

In [209]:
for i in range(len(dataset['horsepower'])):
  if dataset['horsepower'][i]  == '?':
    print("None",i)
    dataset['horsepower'][i] = np.nan
  


None 32
None 126
None 330
None 336
None 354
None 374


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [210]:
dataset.isna().sum()

mpg             0
cylinders       0
displacement    0
horsepower      6
weight          0
acceleration    0
model year      0
origin          0
car name        0
dtype: int64

In [211]:
dataset.dropna(inplace = True)

In [212]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           392 non-null    float64
 1   cylinders     392 non-null    int64  
 2   displacement  392 non-null    float64
 3   horsepower    392 non-null    object 
 4   weight        392 non-null    int64  
 5   acceleration  392 non-null    float64
 6   model year    392 non-null    int64  
 7   origin        392 non-null    int64  
 8   car name      392 non-null    object 
dtypes: float64(3), int64(4), object(2)
memory usage: 30.6+ KB


In [213]:
horse = dataset['horsepower']

In [214]:
horse = np.array(horse)

In [215]:
for i in range(len(horse)):
  horse[i] = int(horse[i])

In [216]:
horse = horse.astype('int')

In [217]:
horse = pd.DataFrame(horse)

In [218]:
type(horse)

pandas.core.frame.DataFrame

In [219]:
print(horse)

       0
0    130
1    165
2    150
3    150
4    140
..   ...
387   86
388   52
389   84
390   79
391   82

[392 rows x 1 columns]


In [227]:
dataset.drop('horsepower', axis = 1, inplace = True)

In [228]:
dataset = pd.concat([dataset, horse], axis = 1)

In [222]:
len(dataset)

392

In [223]:
len(horse)

392

In [229]:
dataset.isna().sum()

mpg             6
cylinders       6
displacement    6
weight          6
acceleration    6
model year      6
origin          6
car name        6
0               6
dtype: int64

In [230]:
dataset.dropna(inplace = True)

In [235]:
dataset.head()

,mpg,cylinders,displacement,weight,acceleration,model year,origin,car name,0
0,18.0,8.0,307.0,3504.0,12.0,70.0,1.0,chevrolet chevelle malibu,130.0
1,15.0,8.0,350.0,3693.0,11.5,70.0,1.0,buick skylark 320,165.0
2,18.0,8.0,318.0,3436.0,11.0,70.0,1.0,plymouth satellite,150.0
3,16.0,8.0,304.0,3433.0,12.0,70.0,1.0,amc rebel sst,150.0
4,17.0,8.0,302.0,3449.0,10.5,70.0,1.0,ford torino,140.0


In [244]:
dataset.rename(columns = {0:"horsepower"}, inplace = True)

In [245]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 386 entries, horsepower to 391
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           386 non-null    float64
 1   cylinders     386 non-null    float64
 2   displacement  386 non-null    float64
 3   weight        386 non-null    float64
 4   acceleration  386 non-null    float64
 5   model year    386 non-null    float64
 6   origin        386 non-null    float64
 7   car name      386 non-null    object 
 8   horsepower    386 non-null    float64
dtypes: float64(8), object(1)
memory usage: 30.2+ KB


In [247]:
dataset.drop('car name', inplace = True, axis = 1)

In [248]:
dataset.describe()

,mpg,cylinders,displacement,weight,acceleration,model year,origin,horsepower
count,386.000000,386.000000,386.000000,386.000000,386.000000,386.000000,386.000000,386.000000
mean,23.320725,5.494819,195.459845,2983.686528,15.505440,75.886010,1.582902,104.559585
std,7.765096,1.709021,105.092187,853.830503,2.721057,3.634247,0.808794,38.728731
min,9.000000,3.000000,68.000000,1613.000000,8.000000,70.000000,1.000000,46.000000
25%,17.000000,4.000000,105.000000,2223.750000,13.725000,73.000000,1.000000,75.000000
50%,22.150000,4.000000,151.000000,2822.500000,15.500000,76.000000,1.000000,93.500000
75%,29.000000,8.000000,302.000000,3627.500000,17.000000,79.000000,2.000000,128.000000
max,46.600000,8.000000,455.000000,5140.000000,24.800000,82.000000,3.000000,230.000000


In [ ]:
#Splitting the dataset
